# Code and Problem Diary

### 17/04/2018

**Description of problem:**
Main issue trying to sort out at a high level is the training algorithm not converging... Assuming the algorithm is implemented correctly then there are two levers by which to alter the convergence of the algorith - the $\tau$ parameter controlling level of sparsity induced by the soft thresholding algorithm and the learning rate of the SGD controlling the step take at each iteration to update the dictionary. Currently we observe the following:
- If we set both $\tau$ and the learning rate to 1 then in the first iteration the sparsity level almost instantly falls to zero. Essentially the l1 error significance is so much larger than the l2 error that the minimum is found by selecting the minimiser of the l1 norm, the zero vector. If this happens in the first iteration then nothing else really matters, our algorithm has reached an equilibrium point from which it cannot escape.
- With a $\tau$ of order $10^{-5}$ and the learning rate still at 1 we finally get a setup where we do not almost immediatly plunge towards the 0 vector. Here we can observe that the l1 error even at 50% sparsity is around 390k while the l2 error term is only around 89k. Hence it is clear why the algorithm is prioritising minimising the l1 error. Interestingly after just two iterations of the whole algorith, e.g. FISTA then dictionary update we reach another equilibrium point, the sparsity level stays at around 63%, while the l2 error increases from around 87k untill it hits around 92k. This implies the learning rate is too high. The stepsize also becomes increasingly small which implies that since $\tau$ is fixed, 1/L is getting smaller and that hence L is getting large. This means that the largest singular value of the weight matrix is getting increasingly large, perhaps implying that the coefficient values are becoming increasingly large in the weight matrix. This is further evidence that the learning rate is to high and suggests and interestingly that the learning rate needs to be low enough to ensure that L is not to large. Furthermore it might suggest we should do something in terms of normalising the kernel filters in some way. This though one would think interrupts with the parameter learning
- Following on from the following bullet point if we use $\tau$ of order $10^{-5}$ and the learning rate at 0.0001 then the average loss per data point and the loss at each iteration at FISTA sticks. Although one would expect the l2 error to stay the same or maybe even increase within each FISTA step one would expect the error between each round of FISTA to decrease, since between each round the dictionary update should be working exclusively to reduce this error. However clearly the dictionary is failing to adapt to reduce the l2 error and converge to a lower value even with a very small step size. If we boost the learning rate to something silly like 10 then we instantly get back to the state we were in before where we sit at around 63% sparsity but with an l2 error of around 121k.

Big problem then seems to be in the dictionary update rather than the FISTA step. Observing the weight update it is clear that the weights are updating overtime, and also as might have been feared, that both $D$ and $D^T$ are consistent. Of note however is that the magnitudes of the weight elements is very small, order $10^{-5}$. Interestingly with tau set to 0, i.e. with no sparisty induced, neither the FISTA or the dictionary update reduce the l2 error term. This is quite curious - intuitively perhaps if the values of the weight matrix are very off then it will make it hard to learn a better X and likewise the converse. However this is far from a rigorously thought through statement! It seems unlikely that Pytorch would incorrectly implement e.g. SGD and code logic seems reasonable, so most likely case is that the FISTA implementation is not correct or we need to use some form of backtracking to update tau to deal with the change in the size of the norm. Next steps:
- Implement IHT with k = 20 and see if this works any better
- On FISTA try normalising the norms so that they are of equal importance/ controlled fully by $\tau$

